In [1]:
pip install ipywidgets langchain langchain-community langchain-core langchainhub tiktoken chromadb pysqlite3-binary sentence-transformers pypdf deepsparse-nightly[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 89.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.8/177.8 kB 200.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 92.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 147.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 218.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 133.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 76.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 166.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 194.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 162.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 186.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Set Up The Model
In this block, we install chromadb and other dependancies.  Chroma requires sqlite3 so that is imported as well.

The LLM that is used is Mistral:Instruct that is hosted by an Ollama container running in OpenShift.

HuggingFace Embeddings are used since they can be run locally and can be configured to take advantage of available GPUs.

In [3]:
__import__('pysqlite3')
import sys
sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')
import chromadb

import bs4
import os.path
from langchain import hub
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.chat_models import ChatOllama
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pprint import pprint
from deepsparse import TextGeneration

#model = ChatOllama(model="mistral:instruct",
#                   base_url="http://ollama-api-service.ollama-llm.svc.cluster.local:11434",
#                   temperature = 0)
model = TextGeneration(model="zoo:llama2-7b-gsm8k_llama2_pretrain-pruned50_quantized")

from langchain_community.embeddings import HuggingFaceEmbeddings

#embd = HuggingFaceEmbeddings(model_kwargs={'device': 'cuda'})
embd = HuggingFaceEmbeddings(model_kwargs={'device': 'cpu'})

[Errno 28] No space left on device
[Errno 28] No space left on device


KeyboardInterrupt: 

[Errno 28] No space left on device
[Errno 28] No space left on device
[Errno 28] No space left on device
--- Logging error ---
Traceback (most recent call last):
  File "/opt/app-root/lib64/python3.9/site-packages/sparsezoo/utils/download.py", line 330, in execute_job_from_queue
    job.func(**job.func_kwargs, **kwargs)
  File "/opt/app-root/lib64/python3.9/site-packages/sparsezoo/utils/download.py", line 383, in download_file
    f.write(chunk)
OSError: [Errno 28] No space left on device

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib64/python3.9/logging/__init__.py", line 1086, in emit
    stream.write(msg + self.terminator)
  File "/opt/app-root/lib64/python3.9/site-packages/ipykernel/iostream.py", line 694, in write
    self._schedule_flush()
  File "/opt/app-root/lib64/python3.9/site-packages/ipykernel/iostream.py", line 590, in _schedule_flush
    self.pub_thread.schedule(_schedule_in_thread)
  File "/opt/a

# Gather Data, Chunk it and Store it in the vector store

If the database is not present, then create it by downloading and chunking the files.  If it is present, then just load it.

In [10]:
persist_dir = "db"

check_file = "False"

path = 'db/chroma.sqlite3'

check_file = os.path.isfile(path)

if check_file is False:
    urls = [
        r'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/html-single/performing_a_standard_rhel_9_installation/index',
        r'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/html-single/performing_an_advanced_rhel_9_installation/index',
        r'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/html-single/configuring_basic_system_settings/index',
        r'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/html-single/security_hardening/index',
        r'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/html-single/composing_a_customized_rhel_system_image/index',
        r'https://access.redhat.com/documentation/en-us/red_hat_enterprise_linux/9/html-single/upgrading_from_rhel_8_to_rhel_9/index',
        r'https://www.redhat.com/en/resources/red-hat-enterprise-linux-subscription-guide'
    ]
    
    loader = WebBaseLoader(urls)
    
    docs = loader.load()
    
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1250, chunk_overlap=0)
    
    splits = text_splitter.split_documents(docs)
    
    vectorstore = Chroma.from_documents(documents=splits, embedding=embd, persist_directory="db")

else:
    
    vectorstore = Chroma(persist_directory=persist_dir, embedding_function=embd)
    
retriever = vectorstore.as_retriever()

# Run the RAG

In [16]:
from langchain_core.runnables import RunnableParallel
from typing import List
from langchain_core.documents import Document


# Prompt
rag_template = """
Given a question write an answer.
Use only the supplied source docs.
If you don't know the answer, just say that you don't know.  Do not fake the answer.
If the answer is relevant, then ALWAYS include a "SOURCES" part in your answer.

QUESTION: {question}
=========
{source_docs}
=========
ANSWER: 
"""

rag_prompt = ChatPromptTemplate.from_template(rag_template)

def format_docs(docs: List[Document]) -> str:
    return "\n\n".join(
        f"Content: {doc.page_content}\nSource: {doc.metadata['source']}" for doc in docs
    )

rag_chain_from_docs = (
    RunnablePassthrough.assign(
        source_docs=(lambda x: format_docs(x["source_docs"]))
    )
    | rag_prompt
    | model
    | StrOutputParser()
)

rag_chain = RunnableParallel(
    {
        "source_docs": retriever,
        "question": RunnablePassthrough()
    }
).assign(answer=rag_chain_from_docs)

question = "How does SELinux work?"

results = (rag_chain.invoke(question))
answer = results["answer"]
print(answer)

 SELinux (Security-Enhanced Linux) is a security extension for the Linux operating system that helps protect against unauthorized access to files, processes, and other resources. It uses a Mandatory Access Control (MAC) model, which sets strict access control rules that must be followed by the system and its applications.

SELinux labels are used to identify different types of data and processes, and access control decisions are based on these labels. The labels define the security context for each resource, including the file type, process, or user. SELinux policies specify the allowed access between different security contexts.

When a process attempts to access a resource, SELinux checks the security context of both the process and the resource against the policy rules. If the access is allowed, the operation is permitted; otherwise, it is denied. This helps prevent unauthorized access or modification of sensitive data.

SELinux policies can be managed using various tools like `sets